In [2]:
pip install pygame


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.6 MB 907.3 kB/s eta 0:00:12
   ---------------------------------------- 0.1/10.6 MB 907.3 kB/s eta 0:00:12
   ---------------------------------------- 0.1/10.6 MB 654.9 kB/s eta 0:00:17
    --------------------------------------- 0.1/10.6 MB 708.1 kB/s eta 0:00:15
    --------------------------------------- 0.1/10.6 MB 708.1 kB/s eta 0:00:15
    --------------------------------------- 0.2/10.6 MB 621.6 kB/s eta 0:00:17
    --------------------------------------- 0.3/10.6 MB 747.0 kB/s eta 0:00:14
   - -------------------------------------- 0.4/10.6 MB 930.9 kB/s eta 0:00:11
   - -------------------------------------- 0.4/10.6 MB 930.9 kB/s eta 0:00:11
   -- ------------------------------------- 0.6/10.6 MB 1.2 MB/s eta 0:

<h3><b>Usage<b></b></h3>
    
<h4>1. Initialize Start and End Points:</h4><h5>Left-click on a cell to set it as the start (orange) node. Left-click again on a different cell to set it as the end (turquoise) node.</h5>

<h4>2. Toggle Obstacle Mode:</h4><h5>Press O on the keyboard to toggle Obstacle Mode on/off. When in obstacle mode, left-click on cells to turn them into obstacles (black), creating barriers.</h5>

<h4>3. Start Pathfinding:</h4><h5>Once you have defined both start and end points, press SPACE to begin the A* pathfinding algorithm. The algorithm will attempt to find the shortest path around obstacles from start to end.</h5>

<h4>4. Clear the Grid:</h4><h5>Press C to reset the grid to a blank state, allowing you to start a new pathfinding configuration.</h5>

<H3>COLOUR SCHEME</H3>
<h5>Start Node: Orange <br>
    <br>
End Node: Turquoise<br>
    <br>
Path Node: Purple<br>
        <br>
Open Node: Light Green<br>
    <br>
Closed Node: Soft Red<br>
    <br>
Obstacle Node: Dark Grey<br>
    <br>
Grid Lines: Light Grey</h4>

In [ ]:
import pygame
import math
from queue import PriorityQueue

pygame.init()
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("A* Path Finding Algorithm with Obstacle Mode")

#Color Scheme
SOFT_RED = (239, 83, 80)
SOFT_GREEN = (129, 199, 132)
SOFT_BLUE = (100, 181, 246)
SOFT_YELLOW = (255, 241, 118)
SOFT_WHITE = (250, 250, 250)
SOFT_BLACK = (55, 71, 79)
SOFT_PURPLE = (186, 104, 200)
SOFT_ORANGE = (255, 183, 77)
SOFT_GREY = (207, 216, 220)
SOFT_TURQUOISE = (128, 222, 234)

class Spot:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = SOFT_WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows

    def get_pos(self):
        return self.row, self.col

    def is_closed(self):
        return self.color == SOFT_RED

    def is_open(self):
        return self.color == SOFT_GREEN

    def is_barrier(self):
        return self.color == SOFT_BLACK

    def is_start(self):
        return self.color == SOFT_ORANGE

    def is_end(self):
        return self.color == SOFT_TURQUOISE

    def reset(self):
        self.color = SOFT_WHITE

    def make_start(self):
        self.color = SOFT_ORANGE

    def make_closed(self):
        self.color = SOFT_RED

    def make_open(self):
        self.color = SOFT_GREEN

    def make_barrier(self):
        self.color = SOFT_BLACK

    def make_end(self):
        self.color = SOFT_TURQUOISE

    def make_path(self):
        self.color = SOFT_PURPLE

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

    def update_neighbors(self, grid):
        self.neighbors = []
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier():
            self.neighbors.append(grid[self.row + 1][self.col])
        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier():
            self.neighbors.append(grid[self.row - 1][self.col])
        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier():
            self.neighbors.append(grid[self.row][self.col + 1])
        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier():
            self.neighbors.append(grid[self.row][self.col - 1])

    def __lt__(self, other):
        return False

def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()

def algorithm(draw, grid, start, end):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {spot: float("inf") for row in grid for spot in row}
    g_score[start] = 0
    f_score = {spot: float("inf") for row in grid for spot in row}
    f_score[start] = h(start.get_pos(), end.get_pos())

    open_set_hash = {start}

    while not open_set.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

        current = open_set.get()[2]
        open_set_hash.remove(current)

        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True

        for neighbor in current.neighbors:
            temp_g_score = g_score[current] + 1

            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()

        draw()

        if current != start:
            current.make_closed()

    return False

def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Spot(i, j, gap, rows)
            grid[i].append(spot)
    return grid

def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, SOFT_GREY, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, SOFT_GREY, (j * gap, 0), (j * gap, width))

def draw(win, grid, rows, width):
    win.fill(SOFT_WHITE)
    for row in grid:
        for spot in row:
            spot.draw(win)
    draw_grid(win, rows, width)
    pygame.display.update()

def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos
    row = y // gap
    col = x // gap
    return row, col

def main(win, width):
    ROWS = 25  # Updated to 25 rows for a smaller grid
    grid = make_grid(ROWS, width)

    start = None
    end = None
    obstacle_mode = False

    run = True
    while run:
        draw(win, grid, ROWS, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if pygame.mouse.get_pressed()[0]:  # LEFT mouse button
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                if obstacle_mode:
                    if spot != start and spot != end:
                        spot.make_barrier()
                elif not start and spot != end:
                    start = spot
                    start.make_start()
                elif not end and spot != start:
                    end = spot
                    end.make_end()

            elif pygame.mouse.get_pressed()[2]:  # RIGHT mouse button
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                spot.reset()
                if spot == start:
                    start = None
                elif spot == end:
                    end = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start and end:
                    for row in grid:
                        for spot in row:
                            spot.update_neighbors(grid)
                    algorithm(lambda: draw(win, grid, ROWS, width), grid, start, end)

                if event.key == pygame.K_c:
                    start = None
                    end = None
                    grid = make_grid(ROWS, width)

                if event.key == pygame.K_o:
                    obstacle_mode = not obstacle_mode
                    print(f"Obstacle Mode: {'ON' if obstacle_mode else 'OFF'}")

    pygame.quit()

main(WIN, WIDTH)